In [1]:
## Connecting to THINKFULs PostgreSQL database

import psycopg2
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [22]:
# logining in -- have to figure a better way to do this
database = 'lifeexpectancy'
username = 'dabc_student'
password = '7*.8G9QH21'

conn = psycopg2.connect(host= '142.93.121.174', port=5432, database=database, user=username, password=password)

# creating a cursor object
cur = conn.cursor()

# querying from database directly into a dataframe
# first we will loop to make sure not in cwd
cwd = os.getcwd()
tables_folder = cwd + '\\Tables'

# here we will select the tables we want based on what is in the sql database
sql_tables = ['lifeexpectancy']
tables =['lifeexpectancy.csv']
if os.path.exists(tables_folder): # creating the tables folder if it doesnt exist
    for table in sql_tables:
        if table not in os.listdir(tables_folder):
            try:
                temp_df = pd.read_sql_query( '''SELECT * FROM {}'''.format(table), conn);
                temp_df.to_csv(tables_folder +'\\' + '{}.csv'.format(table), index= False)     
            except Exception as e:
                print(str(e))
else:
    os.mkdir(tables_folder)
    for table in sql_tables:
        if table not in os.listdir(tables_folder):
            try:
                temp_df = pd.read_sql_query( '''SELECT * FROM {}'''.format(table), conn);
                temp_df.to_csv(tables_folder +'\\' + '{}.csv'.format(table), index= False)    
            except Exception as e:
                print(str(e))    
        
cur.close()
conn.close()   


In [23]:
# loading the dataframes 
lifeexpectancy = pd.read_csv(tables_folder +'\\' + tables[0])
# # finance_df = pd.read_csv(tables_folder +'\\' + tables[1])

print ('The {} DataFrame has {} rows, and {} columns'.format(tables[0], naep_df.shape[0], naep_df.shape[1]))
# display(naep_df.head())

print ('The {} DataFrame has {} rows, and {} columns'.format(tables[1], finance_df.shape[0], finance_df.shape[1]))
display(finance_df.head(lifeexpectancy))

The lifeexpectancy.csv DataFrame has 561 rows, and 8 columns


IndexError: list index out of range

In [20]:
## pre-processing the data
def clean_data(df):
    print(df.isnull().sum())
    
clean_data(finance_df)


Unnamed: 0                       0
capital_outlay_expenditure       0
enroll                          51
federal_revenue                  0
id                               0
instruction_expenditure          0
local_revenue                    0
other_expenditure               51
state                            0
state_revenue                    0
support_services_expenditure     0
total_expenditure                0
total_revenue                    0
year                             0
dtype: int64


In [55]:
os.listdir(tables_folder)

['finance.csv', 'lifeexpectancy.csv', 'naep.csv', 'state_stat.csv']

In [53]:
display(lifeexpectancy.head())
# lifeexpectancy.dtypes

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


Country                             object
Year                                 int64
Status                              object
Life expectancy                    float64
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
dtype: object

In [57]:
# here we will group by Counry and year and take the average then reset_index() so we can further break down
country_LE = lifeexpectancy.groupby(['Country','Year']).mean().reset_index()
display(country_LE.head())
# country_LE.dtypes

# from quick observation we are missing text fields such as Status so we can append those we can now
country_LE['Status'] = lifeexpectancy['Status']

,Country,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2000,54.8,321.0,88,0.01,10.424960,62.0,6532,12.2,...,24.0,8.20,24.0,0.1,114.560000,293756.0,2.3,2.5,0.338,5.5
1,Afghanistan,2001,55.3,316.0,88,0.01,10.574728,63.0,8762,12.6,...,35.0,7.80,33.0,0.1,117.496980,2966463.0,2.1,2.4,0.340,5.9
2,Afghanistan,2002,56.2,3.0,88,0.01,16.887351,64.0,2486,13.0,...,36.0,7.76,36.0,0.1,187.845950,21979923.0,19.9,2.2,0.341,6.2
3,Afghanistan,2003,56.7,295.0,87,0.01,11.089053,65.0,798,13.4,...,41.0,8.82,41.0,0.1,198.728544,2364851.0,19.7,19.9,0.373,6.5
4,Afghanistan,2004,57.0,293.0,87,0.02,15.296066,67.0,466,13.8,...,5.0,8.79,5.0,0.1,219.141353,24118979.0,19.5,19.7,0.381,6.8


Index(['Country', 'Year', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling', 'Status'],
      dtype='object')

,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
Country,,,,,,,,,,,,,,,,,,,
Afghanistan,58.19375,269.0625,78.2500,0.014375,34.960110,64.562500,2362.2500,15.51875,107.5625,48.3750,8.252500,52.3125,0.10000,340.015425,9.972260e+06,16.58125,15.58125,0.415375,8.21250
Albania,75.15625,45.0625,0.6875,4.848750,193.259091,98.000000,53.3750,49.06875,0.9375,98.1250,5.945625,98.0625,0.10000,2119.726679,6.969116e+05,1.61875,1.70000,0.709875,12.13750
Algeria,73.61875,108.1875,20.3125,0.406667,236.185241,78.000000,1943.8750,48.74375,23.5000,91.7500,4.604000,91.8750,0.10000,2847.853392,2.164983e+07,6.09375,5.97500,0.694875,12.71250
Angola,49.01875,328.5625,83.7500,5.740667,102.100268,70.222222,3561.3125,18.01875,132.6250,46.1250,3.919333,47.6875,2.36875,1975.143045,1.014710e+07,6.19375,6.66875,0.458375,8.04375
Antigua and Barbuda,75.05625,127.5000,0.0000,7.949333,1001.585226,98.266667,0.0000,38.42500,0.0000,96.9375,4.791333,98.3125,0.12500,9759.305728,NaN,3.42500,3.37500,0.488625,8.84375
